In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Loading the images

In [3]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (128,128))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('dataset')
print('loaded',len(samples),' samples')

labels[labels=='new_cats']='cats'
labels[labels=='new_dogs']='dogs'

print('classes',set(labels))

# normalization
samples = samples / 255.0

loaded 6192  samples
classes {'cats', 'dogs'}


# One-hot encoding

In [4]:
from sklearn.preprocessing import LabelEncoder 

encoder = LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels.shape)

One-hot labels (6192, 2)


In [5]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLables) = sklearn.model_selection.train_test_split(samples,labels,random_state=42)

# Creating the model

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(128,128,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(300))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 activation (Activation)     (None, 128, 128, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 activation_1 (Activation)   (None, 64, 64, 32)        0         
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                        

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [8]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS = 30
BATCH_SIZE = 100
checkpt = ModelCheckpoint (filepath='model.h5', save_best_only=True)
estopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

model.fit(trainSamples, trainLabels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(testSamples, testLables), callbacks=[checkpt, estopping])

Epoch 1/30
47/47 [==============================] - 47s 974ms/step - loss: 0.6912 - accuracy: 0.5950 - val_loss: 0.6270 - val_accuracy: 0.6382
Epoch 2/30
47/47 [==============================] - 41s 872ms/step - loss: 0.5058 - accuracy: 0.7556 - val_loss: 0.4540 - val_accuracy: 0.8101
Epoch 3/30
18/47 [==========>...................] - ETA: 22s - loss: 0.3884 - accuracy: 0.8256

KeyboardInterrupt: 

# Results

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
testResults = model.predict(testSamples)

print(confusion_matrix(testLables.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(testLables.argmax(axis=1), testResults.argmax(axis=1)))
print("CNN Accuracy: {:.2f}".format(accuracy_score(testLables.argmax(axis=1), testResults.argmax(axis=1))))
print("Cohen's Kappa: {:.2f}".format(cohen_kappa_score(testLables.argmax(axis=1), testResults.argmax(axis=1))))

49/49 [==============================] - 3s 54ms/step
[[746  31]
 [ 55 716]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       777
           1       0.96      0.93      0.94       771

    accuracy                           0.94      1548
   macro avg       0.94      0.94      0.94      1548
weighted avg       0.94      0.94      0.94      1548

CNN Accuracy: 0.94
Cohen's Kappa: 0.89


In [13]:
tf.saved_model.save(model,'model')

INFO:tensorflow:Assets written to: model\assets


In [11]:
from tensorflow.python.keras.models import load_model
model = load_model("final_model.h5") ### add the correct model name!!!
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
activation_12 (Activation)   (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
activation_13 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)       